---

thoughts: what is the fundamental question with respect to categorisation? is learning a representation the most important part? but once we have a learned representation we can still endlessly argue about how humans really categorise concepts; are we conjuring in our heads a single prototypical representation based on past experience or are we referring to every existing exemplar we have previously encountered? or is there a statistical model that encapsulates both by forming decision boundaries -- and is this what we should use to model all (?) categories in existence. 

---

similarity metrics: what kind of similarity metric in the same space allows you to best determine how similar two vector representations (of categories) are?
how does this change if your representations don't share the same space (this part is mostly testing metrics, but important)?

---

what objectives do we want to train on that best allows us to represent a category the way humans do? 

---

once we have a representation of a concept and what we want to categorise, what is the best way to further generalize to other concepts? how do humans do this? are we conjuring in our minds a prototypical representation of the category based on previous encounters with instances from this category? 

---

can humans learn from sketches and more abstract representations better than photos? children can replicate concepts through sketches -- does training on photos and testing on sketches / allowing generalization through sketches make more sense wrt human learning?

---

given that sketches are more abstract than colored, pixelated photo images -- does a sketch representation of a category align with deeper layers of a neural network trained on images? and if this is true, is this what is helping them generalise?

---


In [20]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

path = os.getcwd(); cat_path = '/'.join(i for i in path.split('/')[:-1]) + '/proto/data/sketchy/ecategories/'
# path to random experimental 10 images for each category
categories = os.listdir(cat_path)

# shows the different categories in the Sketchy dataset (categories vary for different datasets)
def print_category_names():
    print 'Length: ' + str(len(categories))
    print sorted(set(categories))

    
# show 'num' 'img_type' images for a category 'category' 
# e.g., show_category_images('cat', 'sketch', 5) or show_category_images('cat', 'photo', 10)
def show_category_images(category, img_type, num):
    assert num < 11
    assert os.path.isdir(cat_path + category)
    f = open(cat_path + category + '/' + img_type + '/filenames.txt')
    fnames = [line.strip() for line in f.readlines()[1:]]
    for fname in fnames[:num]:
        fpath = path + '/data/sketchy/efigs/' + fname
        if os.path.isfile(fpath) is False: continue
        img = Image.open(fpath)
        img = np.array(img).astype(np.uint8)
        print np.sum(img)
        print 'category: ' + str(category)
        plt.imshow(img); plt.show()
      
    
# helper function
def get_category_average(category, img_type, num):
    assert num < 11
    assert os.path.isdir(cat_path + category)
    f = open(cat_path + category + '/' + img_type + '/filenames.txt')
    fnames = [line.strip() for line in f.readlines()[1:]]
    avg = np.zeros((256, 256, 3))
    for fname in fnames[:num]:
        fpath = path + '/data/sketchy/efigs/' + fname
        if os.path.isfile(fpath) is False: continue
        img = Image.open(fpath)
        avg += np.array(img).astype(np.uint8)
    avg = np.array(avg / (1.0*num)).astype(np.uint8)
    return avg

# show average averaged over 'num' images for category 'category' 
# e.g., show_category_average('dog', 'sketch', '4')
def show_category_average(category, img_type, num):
    avg = get_category_average(category, img_type, num)
    print 'just your average ' + str(category)
    plt.imshow(avg); plt.show()
    

from sklearn.metrics.pairwise import cosine_similarity as cos_sim
from scipy.stats import spearmanr, entropy
from sklearn.metrics import log_loss, mutual_info_score
from skimage import data, io, filters, color
from PIL import ImageOps
from sklearn.decomposition import PCA
max_intensity = 255.0

def similarity_metrics(img_1, img_2):
    #plt.imshow(img_1); plt.show()
    #plt.imshow(img_2); plt.show()
    
    #img_1 = img_1.resize((16, 16)); img_2 = img_2.resize((16, 16))
    #img_1, img_2 = ImageOps.invert(img_1), ImageOps.invert(img_2)
    img_1 = np.array(img_1).astype(np.uint8)
    img_2 = np.array(img_2).astype(np.uint8)
    # grayscale
    img_1 = color.rgb2gray(img_1)
    img_2 = color.rgb2gray(img_2)
    # reduce dimensions
    n_components = 3
    pca = PCA(n_components=n_components); 
    m1 = pca.fit_transform(img_1); m2 = pca.fit_transform(img_2)
    # flatten
    m1, m2 = [item for sublist in m1 for item in sublist], [item for sublist in m2 for item in sublist]
    # make intensities between 0-1
    m1, m2 = m1 / np.sum([max_intensity]), m2 /  np.sum([max_intensity])
    # normalise
    #m1, m2 = m1 / np.sum(m1), m2 / np.sum(m2)

    #plt.imshow(img_2, cmap='gray'); plt.show()
    metric_names = ['cos_sim', 'spearmanr', 'abs_diff', 'sq_diff', 'kl', 'mutual_inf']; 
    metrics = {'cos_sim': cos_sim(np.asarray([m1]), np.asarray([m2]))[0][0],
               'spearmanr': spearmanr(m1, m2)[0],
               'abs_diff': np.sum((m1.astype("float") - m2.astype("float"))),
               'sq_diff:': np.sum((m1.astype("float") - m2.astype("float")) ** 2) / float(m1.shape[0] * m2.shape[0])
    }
    return metrics

def get_similarity_1(category_1, category_2, img_type):
    avg_1 = get_category_average(category_1, img_type, 5)
    avg_2 = get_category_average(category_2, img_type, 5)
    img_1, img_2 = Image.fromarray(avg_1), Image.fromarray(avg_2)
    metrics = similarity_metrics(img_1, img_2)
    print metrics

def get_similarity_2(category_1, img_type_1, category_2, img_type_2):
    avg_1 = get_category_average(category_1, img_type_1, 5)
    avg_2 = get_category_average(category_2, img_type_2, 5)
    img_1, img_2 = Image.fromarray(avg_1), Image.fromarray(avg_2)
    metrics = similarity_metrics(img_1, img_2)
    print metrics

def get_image(category, img_type):
    assert os.path.isdir(cat_path + category)
    f = open(cat_path + category + '/' + img_type + '/filenames.txt')
    fnames = [line.strip() for line in f.readlines()[1:]]
    np.random.shuffle(fnames); fpath = path + '/data/sketchy/efigs/' + fnames[0]
    img = Image.open(fpath)
    return img

def get_similarity_3(category, img_type):
    avg = get_category_average(category, img_type, 5)
    avg = Image.fromarray(avg)
    img = get_image(category, img_type)
    metrics = similarity_metrics(img, avg)
    print metrics
  
##
import pandas
import seaborn as sns
# takes in a list of categories and generates a map of similarities
# categories = ['cat', 'dog', 'mouse', 'rabbit', 'table', 'couch', 'duck', 'swan', 'penguin', 'dolphin']
# e.g., similarity_map(categories, 'sketch', 'cos_sim')
def similarity_map(categories, img_type, metric):
    sim = np.zeros((len(categories), len(categories)))
    for i in range(len(categories)):
        for j in range(len(categories)):
            avg_1 = get_category_average(categories[i], img_type, 5)
            avg_2 = get_category_average(categories[j], img_type, 5)
            metrics = similarity_metrics(Image.fromarray(avg_1), Image.fromarray(avg_2))
            if metric not in metrics.keys(): continue
            sim[i][j] = metrics[metric]
            
    sns.set()
    df = pandas.DataFrame(sim)
    df.columns = categories; df.index = categories
    ax = sns.heatmap(df, cmap="YlGnBu", annot=True)
    ax.set_ylabel(img_type + '_' + metric)
    plt.show()
    
def test():
    img_1 = Image.open(path + '/data/sketchy/efigs/n02041246_1477-3.png'); 
    img_2 = Image.open(path + '/data/sketchy/efigs/n02041246_1477-5.png'); 
    img_3 = Image.open(path + '/data/sketchy/efigs/n02432983_20607.jpg'); 
    img_4 = Image.open(path + '/data/sketchy/efigs/n02431337_16475.jpg'); 

    metrics = similarity_metrics(img_1, img_2)
    print metrics
    metrics = similarity_metrics(img_1, img_3)
    print metrics
    metrics = similarity_metrics(img_4, img_3)
    print metrics
 



In [ ]:
show_category_images('airplane', 'photo', 2)
show_category_average('airplane', 'photo', 5)
show_category_images('bicycle', 'sketch', 2)
show_category_average('bicycle', 'sketch', 5)

show_category_images('wine_bottle', 'photo', 2)
show_category_average('wine_bottle', 'sketch', 5)
show_category_average('dog', 'sketch', 5)



Takeaway from results:

given (1) doesn't work, we can't learn from the more abstract, but can use it to generalise? 

does evaluating at every layer give us more insight into this? 

In [21]:
# read Tom Griffiths "human-eval", Beth Levin, Trevor Darrell (?) and other vision folks